In [1]:
import pandas as pd
import numpy as np
import math 
from casadi import *
from pylab import *
import os 
from datetime import datetime, timedelta
import pymap3d # python3 -m pip install pymap3d
import random
import pickle
from scipy.special import lambertw
from scipy import stats
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline
from bokeh.plotting import figure, show, output_notebook, output_file, reset_output, save
from bokeh.layouts import gridplot, row, column
from bokeh.models import Range1d
output_notebook() 

Loading BokehJS ...

In [2]:
nbuf = 30

# Functions to use

In [3]:
def system_update(system,x,y,b):
    dict = {'X': x, 'Y': y, 'N': b}
    system = system.append(dict, ignore_index = True).drop(index=[0]).reset_index(drop=True)
    return system

def privacy(system):
    n = system['N'].sum()
    if n == 0:
        barx = 0
        bary = 0
        priv = 0
    else:
        barx = np.dot(system['X'], system['N'])/n
        bary = np.dot(system['Y'], system['N'])/n
        #priv = np.sqrt(np.dot((system['X'] - barx)**2 + (system['Y'] - bary)**2, system['N'])/n) 
        priv = np.dot(np.sqrt((system['X'] - barx)**2 + (system['Y'] - bary)**2), system['N'])/n 
    return barx, bary, priv

def utility(x,y):
    return np.sqrt(x**2 + y**2)

def reset_system(system):
    n = len(system)
    X = np.array([0]*n)
    Y = np.array([0]*n)
    N = np.array([0]*n)

    system['X'] = X
    system['Y'] = Y
    system['N'] = N
    return system



def GeoI_LPPM(x,y,epsilon):
    theta_GeoI = random.uniform(0,2*math.pi)
    r_GeoI = -1/epsilon*(np.real(lambertw((random.random()-1)/math.exp(1),k=-1))+1);
    x_ctrl = x + r_GeoI*math.cos(theta_GeoI)
    y_ctrl = y + r_GeoI*math.sin(theta_GeoI)
    return x_ctrl, y_ctrl, r_GeoI 
    

## Funciones simbólicas

In [4]:
from sys import path

# Control
u = MX.sym("u")
v = MX.sym("v")
b = MX.sym("b")

# State
x = MX.sym("x",nbuf)
y = MX.sym("y",nbuf)
nx = MX.sym("nx",nbuf)

# Matrices
A = np.zeros((nbuf, nbuf))
B = np.zeros((nbuf, 1))
for i in range(nbuf-1):
    A[i,i+1] = 1

B[nbuf-1,0] = 1
A.tolist()
B.tolist()
A = DM(A)
B = DM(B)

# Dynamics
xplus = A@x + B@u
yplus = A@y + B@v
nplus = A@nx + B@b

# Discrete time dynamics function
F = Function('F', [x,u],[xplus])

#Privacity function
O = DM(np.ones((1,nbuf)))
xbar = (O@(x*nx))/((O@nx)+1e-8)
ybar = O@(y*nx)/((O@nx)+1e-8)
P = (O@( sqrt((x-xbar)**2 + (y-ybar)**2)*nx) )/((O@nx)+1e-8)
J = Function('J', [x,y,nx],[xbar, ybar, P],['x','y','nx'],['xbar', 'ybar','P'])

# Test privacy functions
x1 = np.random.normal(loc=0.0, scale=0.35, size=(1,nbuf)).tolist()[0]
y1 = np.random.normal(loc=0.0, scale=0.35, size=(1,nbuf)).tolist()[0]
nx1 = [1]*nbuf

## GeoI

In [5]:
#Generating Geo-I data
def func_run(user_n,T_i,T_f,horizons):
    user= str(user_n) # Examples: abboip oilrag 51 90 14
    print(user)
    local_url="datasets/"+dataset+"-tree/"+user+".csv"
    sol_file='solutions/'+dataset+'_30s/GeoI_10000/sol_' +str(user_n)
    exists = os.path.isfile(local_url)
    if exists:
        data = pd.read_csv(local_url,names=["latitude","longitude","timestamp"])
    else:
        return #if user doesn't exist return out of the function in order to get another user


    if not os.path.exists(sol_file):
        os.makedirs(sol_file)

    data['time']=[datetime.utcfromtimestamp(tstamp/1000) for tstamp in data['timestamp']]
    data['elapsedtime']=(data['timestamp']-data['timestamp'][0])/1000
    whole_duration = data['elapsedtime'][len(data['timestamp'])-1]/60


    data=data.set_index('time').sort_index().copy()
    ell_grs80 = pymap3d.Ellipsoid('grs80') 
    data['altitude']=np.zeros(len(data.index))
    lat0, lon0, h0 = data['latitude'][0],data['longitude'][0],data['altitude'][0]
    data['x'], data['y'], data['u_enu'] = pymap3d.geodetic2enu(data['latitude'], data['longitude'], data['altitude'], lat0, lon0, h0, ell=ell_grs80)
    data.head()

    # Real transmitted data
    Tmin = T_i
    Tmax = T_f
    # Tmax = data['elapsedtime'][-1]
    xr = data['x'][(data['elapsedtime'] >= Tmin) & (data['elapsedtime'] < Tmax)]
    yr = data['y'][(data['elapsedtime'] >= Tmin) & (data['elapsedtime'] < Tmax)]
    ur = data['u_enu'][(data['elapsedtime'] >= Tmin) & (data['elapsedtime'] < Tmax)]
    Timer = data['elapsedtime'][(data['elapsedtime'] >= Tmin) & (data['elapsedtime'] < Tmax)]

    # Resampling the data with sample time ts, averaging over the intervals
    ts = 10  # Sampling time
    tmax = Timer[-1]  # Final time
    nt = int(tmax/ts)+1


    newx = []
    newy = []
    newu = []
    newb = []
    newTimer = []
    for i in range(0,nt):
        xt = data['x'][(data['elapsedtime'] >= i*ts) & (data['elapsedtime'] < (i+1)*ts)].mean() 
        yt = data['y'][(data['elapsedtime'] >= i*ts) & (data['elapsedtime'] < (i+1)*ts)].mean() 
        ut = data['u_enu'][(data['elapsedtime'] >= i*ts) & (data['elapsedtime'] < (i+1)*ts)].mean() 
        if np.isnan(xt) == False:
            newx += [xt]
            newy += [yt]
            newu += [ut]
            newb += [1]
        else:  # Fill with zeros when no transmitted
            newx += [0]
            newy += [0]
            newu += [0]
            newb += [0]
        newTimer += [(i+1)*ts]

    xs = np.array(newx)
    ys = np.array(newy)
    bs = np.array(newb)
    us = np.array(newu)
    Time = np.array(newTimer)


    save_data_raw = pd.DataFrame()
    save_data_raw['Timer'] = Timer
    save_data_raw['xr'] = xr
    save_data_raw['yr'] = yr
    pickle.dump( save_data_raw, open( sol_file+"/save_data_raw.pkl", "wb" ) )
    save_data_raw.to_csv(sol_file+'/save_data_raw.csv')
    
    save_data = pd.DataFrame()
    save_data['Time'] = Time
    save_data['xs'] = xs
    save_data['ys'] = ys
    save_data['us'] = us
    save_data['bs'] = bs
    pickle.dump( save_data, open( sol_file+"/save_data.pkl", "wb" ) )
    save_data.to_csv(sol_file+'/save_data.csv')


    # Vector for plotting, removed the non-transmitted instants
    Timep = Time[bs != 0]
    xsp = xs[bs != 0] 
    ysp = ys[bs != 0] 


    # Create the dynamic system state
    n = 30
    X = np.array([0]*n)
    Y = np.array([0]*n)
    N = np.array([0]*n)

    system = pd.DataFrame()
    system['X'] = X
    system['Y'] = Y
    system['N'] = N

    #Generating Geo-I data
    x_obf=np.zeros_like(xs)
    y_obf=np.zeros_like(ys)
    
    for i in range(nt):
        x_obf[i] ,y_obf[i], r =GeoI_LPPM(xs[i],ys[i],0.03)

    delta=np.zeros((2,nt))
    delta[0,]=x_obf-xs
    delta[1,]=y_obf-ys


    #Simulate real an obfuscated trajectories
    system = reset_system(system)
    barxk = []
    baryk = []
    privk = []
    system_obf = reset_system(system)
    barxk_obf = []
    baryk_obf = []
    privk_obf = []
    util = []

    for i in range(0,nt,1):
        bi = bs[i] 
        # Update the real system
        system = system_update(system,xs[i],ys[i],bi)
        barx, bary, priv = privacy(system)
        barxk += [barx] 
        baryk += [bary] 
        privk += [priv] 
        # Update the obfuscated system
        system_obf = system_update(system_obf,x_obf[i],y_obf[i],bi)
        barx_obf, bary_obf, priv_obf = privacy(system_obf)
        barxk_obf += [barx_obf] 
        baryk_obf += [bary_obf] 
        privk_obf += [priv_obf] 
        # Compute utility
        util += [utility(delta[0,i],delta[1,i])*bi]

    util_array = np.array(util)
    util2 = np.mean(util_array[util_array > 0])


    Data_real = pd.DataFrame()
    Data_real['x'] = xs
    Data_real['y'] = ys
    Data_real['b'] = bs
    Data_real['barx'] = barxk
    Data_real['bary'] = baryk
    Data_real['priv'] = privk
    Data_real['u_enu'] = us

    Data_GeoI = pd.DataFrame()
    Data_GeoI['x'] = x_obf
    Data_GeoI['y'] = y_obf
    Data_GeoI['U'] = delta[0,]
    Data_GeoI['V'] = delta[1,]
    Data_GeoI['barx'] = barxk_obf
    Data_GeoI['bary'] = baryk_obf
    Data_GeoI['priv'] = privk_obf
    Data_GeoI['util'] = util
    Data_GeoI['u_enu'] = us

    pickle.dump( Data_real, open( sol_file+"/Data_real.pkl", "wb" ) )
    Data_real.to_csv(sol_file+"/Data_real.csv")
    pickle.dump( Data_GeoI, open( sol_file+"/Data_GeoI.pkl", "wb" ) )
    Data_GeoI.to_csv(sol_file+"/Data_GeoI.csv")  



In [6]:
dataset="privamov" # OPTIONS ARE: cabspotting privamov
#horizon 7->3min
#hor=[1,2,3,4,5,6,7,8,9,10]
hor=[1,2,3,5,8,13,16]
time_0 = time.monotonic()
for user in range(1,118):
    func_run(user,0,10000,hor)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
